# MLOps workshop with Amazon SageMaker

## Module 02: Transform the data and train a model inside a Jupyter notebook.

In this workshop we will demonstrate a journey to cloud-native machine learning starting from a more traditional approach to model development and training directly in Jupyter notebooks to remote managed data transformations and training with Amazon SageMaker to fully automated pipelines with SageMkaer Pipelines.

In this first notebook we will predict house prices based on the well-known [Boston Housing dataset](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) with a simple regression model in Tensorflow 2. This public dataset contains 13 features regarding housing stock of towns in the Boston area.  Features include average number of rooms, accessibility to radial highways, adjacency to a major river, etc.  

To begin, we'll import some necessary packages and set up directories for training and test data.  We'll also set up a SageMaker Session to perform various operations, and specify an Amazon S3 bucket to hold input data and output.  The default bucket used here is created by SageMaker if it doesn't already exist, and named in accordance with the AWS account ID and AWS Region.  

In [ ]:
!pip install matplotlib seaborn

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
import os

data_dir = os.path.join(os.getcwd(), 'data')
os.makedirs(data_dir, exist_ok=True)

train_dir = os.path.join(os.getcwd(), 'data/train')
os.makedirs(train_dir, exist_ok=True)

test_dir = os.path.join(os.getcwd(), 'data/test')
os.makedirs(test_dir, exist_ok=True)

raw_dir = os.path.join(os.getcwd(), 'data/raw')
os.makedirs(raw_dir, exist_ok=True)

batch_dir = os.path.join(os.getcwd(), 'data/batch')
os.makedirs(batch_dir, exist_ok=True)

# Exploratory Data Analysis (EDA)

According to The [State of Data Science 2020](https://www.anaconda.com/state-of-data-science-2020) survey, data management, exploratory data analysis (EDA), feature selection, and feature engineering accounts for more than 66% of a data scientist’s time.

Exploratory Data Analysis is an approach in analyzing data sets to summarize their main characteristics, often using statistical graphics and other data visualization methods.
EDA assists Data science professionals in various ways:-

- Getting a better understanding of data.
- Identifying various data patterns.
- Getting a better understanding of the problem statement.

Numerical EDA gives you some very important information, such as the names and data types of the columns, and the dimensions of the DataFrame. 
Visual EDA on the other hand will give you insight into features and target relationship and distribution.

First we'll load the Boston Housing dataset and explore the data.

In [ ]:
from tensorflow.python.keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
columns=['CRIM',
         'ZN',
         'INDUS',
         'CHAS',
         'NOX',
         'RM',
         'AGE',
         'DIS',
         'RAD',
         'TAX',
         'PTRATIO',
         'B',
         'LSTAT']

df = pd.DataFrame(x_train, columns=columns)
df["MEDV"] = y_train
df

### Numerical EDA

Check how big is dataset, how many and of what type features it has, and what is target.

In [ ]:
df.info()

There are 14 attributes in each case of the dataset. They are:
1. CRIM - per capita crime rate by town.
2. ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
3. INDUS - proportion of non-retail business acres per town.
4. CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise).
5. NOX - nitric oxides concentration (parts per 10 million).
6. RM - average number of rooms per dwelling.
7. AGE - proportion of owner-occupied units built prior to 1940.
8. DIS - weighted distances to five Boston employment centres.
9. RAD - index of accessibility to radial highways.
10. TAX - full-value property-tax rate per 10K dollars.
11. PTRATIO - pupil-teacher ratio by town.
12. B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.
13. LSTAT - % lower status of the population.
14. MEDV - Median value of owner-occupied homes in $1000's.

Now, let's summarize the data to see the distribution of data

In [ ]:
df.describe()

### Visual EDA

Let's try to find the crime level in relation to citizen status and median value of houses.

In [ ]:
df['medv_bins'] = pd.cut(df.MEDV, bins=5, include_lowest=True)
df.medv_bins.head()

In [ ]:
df['lstat_bins'] = pd.cut(df.LSTAT, bins=[0, 7, 17, 38], labels=['richest', 'ordinary', 'poorest'], include_lowest=True)
df.lstat_bins.head()

In [ ]:
f, (ax_viol, ax_box) = plt.subplots(2, sharex=True)

sns.boxplot(x='medv_bins', y='CRIM', data=df, ax=ax_viol)
sns.swarmplot(x='medv_bins', y='CRIM', hue='lstat_bins', data=df, ax=ax_box)

ax_viol.set(xlabel='')

We can see that the as the citizens status is poorer (LSTAT) and median value of houses is lower (MEDV), crime level are indeed growing (CRIM).

We will stop with EDA for now for the sake of time needed for the rest of the workshop.

# Dataset transformation <a class="anchor" id="SageMakerProcessing">

Next, we'll transform the dataset. In a typical SageMaker workflow, notebooks are only used for prototyping and can be run on relatively inexpensive and less powerful instances, while processing, training and model hosting tasks are run on separate, more powerful SageMaker-managed instances. 

We'll now save the raw feature data, and also save the labels for training and testing.

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

np.save(os.path.join(raw_dir, 'x_train.npy'), x_train)
np.save(os.path.join(raw_dir, 'x_test.npy'), x_test)
np.save(os.path.join(raw_dir, 'y_train.npy'), y_train)
np.save(os.path.join(raw_dir, 'y_test.npy'), y_test)

Next, we'll execute the data preprocessing as shown below.

In [ ]:
import glob
import numpy as np
import os
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
x_train = np.load(os.path.join(raw_dir, 'x_train.npy'))
scaler.fit(x_train)

In [ ]:
input_files = glob.glob('{}/raw/*.npy'.format(data_dir))
print('\nINPUT FILE LIST: \n{}\n'.format(input_files))
for file in input_files:
    raw = np.load(file)
    # only transform feature columns
    if 'y_' not in file:
        transformed = scaler.transform(raw)
    if 'train' in file:
        if 'y_' in file:
            output_path = os.path.join(train_dir, 'y_train.npy')
            np.save(output_path, raw)
            print('SAVED LABEL TRAINING DATA FILE\n')
        else:
            output_path = os.path.join(train_dir, 'x_train.npy')
            np.save(output_path, transformed)
            print('SAVED TRANSFORMED TRAINING DATA FILE\n')
    else:
        if 'y_' in file:
            output_path = os.path.join(test_dir, 'y_test.npy')
            np.save(output_path, raw)
            print('SAVED LABEL TEST DATA FILE\n')
        else:
            output_path = os.path.join(test_dir, 'x_test.npy')
            np.save(output_path, transformed)
            print('SAVED TRANSFORMED TEST DATA FILE\n')

#  Training <a class="anchor" id="SageMakerHostedTraining">

Now that we've prepared a dataset, we can move on to model training.

In [ ]:
import numpy as np
import os
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def get_train_data(train_dir):
    x_train = np.load(os.path.join(train_dir, 'x_train.npy'))
    y_train = np.load(os.path.join(train_dir, 'y_train.npy'))
    print('x train', x_train.shape,'y train', y_train.shape)

    return x_train, y_train


def get_test_data(test_dir):
    x_test = np.load(os.path.join(test_dir, 'x_test.npy'))
    y_test = np.load(os.path.join(test_dir, 'y_test.npy'))
    print('x test', x_test.shape,'y test', y_test.shape)

    return x_test, y_test

def get_model():
    inputs = tf.keras.Input(shape=(13,))
    hidden_1 = tf.keras.layers.Dense(13, activation='tanh')(inputs)
    hidden_2 = tf.keras.layers.Dense(6, activation='sigmoid')(hidden_1)
    outputs = tf.keras.layers.Dense(1)(hidden_2)
    return tf.keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
x_train, y_train = get_train_data(train_dir)
x_test, y_test = get_test_data(test_dir)

device = '/cpu:0'
print(device)
batch_size = 128
epochs = 80
learning_rate = 0.01
print('batch_size = {}, epochs = {}, learning rate = {}'.format(batch_size, epochs, learning_rate))

with tf.device(device):
    model = get_model()
    optimizer = tf.keras.optimizers.SGD(learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
              validation_data=(x_test, y_test))

    # evaluate on test set
    scores = model.evaluate(x_test, y_test, batch_size, verbose=2)
    print("\nTest MSE :", scores)


The unzipped archive should include the assets required by TensorFlow Serving to load the model and serve it, including a .pb file:  

In [ ]:
model.save('model' + '/1')

# Scoring the model

In [ ]:
import numpy as np
import tensorflow as tf

model = tf.keras.models.load_model('model/1')

x_test = np.load(os.path.join(test_dir, 'x_test.npy'))
y_test = np.load(os.path.join(test_dir, 'y_test.npy'))
scores = model.evaluate(x_test, y_test, verbose=2)
print("\nTest MSE :", scores)